In [1]:
import json
import pandas as pd
import mysql.connector
from time import sleep
from google.cloud import secretmanager
from google.cloud import bigquery

In [2]:
# Crea una instancia del cliente de BigQuery
client = bigquery.Client(project="data-development-337318")

In [3]:
get_products = """
SELECT id, name
FROM `data-development-337318.data_standard.app_products_processed_most_trx_products`
"""
df = pd.read_gbq(get_products, dialect='standard', use_bqstorage_api=True, project_id="data-production-337318")
df

,ean13,name
0,3328640122317,HIGHLAND QUEEN
1,7702133815782,Halls Barra Hierbabuena Lyptus
2,7702031902959,GOHNSONS BABY ORIGINAL 75g
3,6950225101874,TOALLA SANITARIA SMILIFE ANION ENJOYABLE NOCHE...
4,6941639823681,TALADRO PERCUTOR BATERIA TOTAL 20V 45NM - TIDL...
...,...,...
5981806,7702024026099,Nescafé Tradición Ristra x 12 Un x 25 gr
5981807,7707007213909,Pague 23 lleve 25 Bolsas de Azúcar Blanco Espe...
5981808,7702024067818,Galleta Milo Sandwich Chocoleche Paquete x 12 ...
5981809,7500435201322,Detergente Ariel Regular Bolsa x 225 gr


In [17]:
def get_match(product_ids):
    query = f"""
    CREATE TEMP FUNCTION calculate_similarity(non_standardized ARRAY<STRUCT<value INT64>>, standardized ARRAY<STRUCT<value INT64>>) AS (
      (SELECT SUM(IF(item1.value = item2.value, 1, 0)) FROM UNNEST(non_standardized) AS item1
        JOIN UNNEST(standardized) AS item2 WITH OFFSET o2 ON o2 BETWEEN 0 AND ARRAY_LENGTH(non_standardized)-1 AND item1.value = item2.value) / ARRAY_LENGTH(non_standardized)
    );

    WITH expanded_valid_ean AS (
      SELECT id, name, name_vector, GENERATE_ARRAY(0, ARRAY_LENGTH(name_vector)-1) AS index, SKU
      FROM `data-development-337318.data_standard.app_products_processed_valid_ean`
    ),
    expanded_invalid_ean AS (
      SELECT id, name, name_vector, GENERATE_ARRAY(0, ARRAY_LENGTH(name_vector)-1) AS index
      FROM `data-development-337318.data_standard.app_products_processed_most_trx_products`
    ),
    similarity_scores AS (
      SELECT ve.id AS id_valid_ean, 
             ve.name AS name_valid_ean,
             ve.SKU,
             ive.id AS id_invalid_ean, 
             ive.name AS name_invalid_ean,
             calculate_similarity(ive.name_vector, ve.name_vector) AS similarity,
             ROW_NUMBER() OVER (PARTITION BY ive.id ORDER BY calculate_similarity(ive.name_vector, ve.name_vector) DESC) AS rn
      FROM expanded_valid_ean ve
      CROSS JOIN expanded_invalid_ean ive  
      WHERE ive.id IN {str(product_ids)}
    )
    SELECT id_valid_ean, name_valid_ean, id_invalid_ean, name_invalid_ean, similarity, SKU
    FROM similarity_scores
    WHERE rn <= 5
    ORDER BY id_invalid_ean, similarity DESC;
    """
    return pd.read_gbq(query, dialect='standard', use_bqstorage_api=True, project_id="data-production-337318")

In [30]:
batch = 10
lng = len(df)

#dfc = pd.DataFrame()
for i in range(35910, lng, batch):
    min_ = i
    max_ = i+batch
    if max_ >= lng:
        max_ = lng-1
    
    print(min_, max_)
    product_ids = tuple(df["id"][min_:max_])
    response = get_match(product_ids)
    
    dfc = pd.concat([dfc,response])

dfc.reset_index(inplace=True, drop=True)
dfc

35910 35920
35920 35930
35930 35940
35940 35950
35950 35960
35960 35970
35970 35980
35980 35990
35990 36000
36000 36010
36010 36020
36020 36030
36030 36040
36040 36050
36050 36060
36060 36070
36070 36080
36080 36090
36090 36100
36100 36110
36110 36120
36120 36130
36130 36140
36140 36150
36150 36160
36160 36170
36170 36180
36180 36190
36190 36200
36200 36210
36210 36220
36220 36230
36230 36240
36240 36250
36250 36260
36260 36270
36270 36280
36280 36290
36290 36300
36300 36310
36310 36320
36320 36330
36330 36340
36340 36350
36350 36360
36360 36370
36370 36380
36380 36390
36390 36400
36400 36410
36410 36420
36420 36430
36430 36440
36440 36450
36450 36460
36460 36470
36470 36480
36480 36490
36490 36500
36500 36510
36510 36520
36520 36530
36530 36540
36540 36550
36550 36560
36560 36570
36570 36580
36580 36590
36590 36600
36600 36610
36610 36620
36620 36630
36630 36640
36640 36650
36650 36660
36660 36670
36670 36680
36680 36690
36690 36700
36700 36710
36710 36720
36720 36730
36730 36740
3674

42740 42750
42750 42760
42760 42770
42770 42780
42780 42790
42790 42800
42800 42810
42810 42820
42820 42830
42830 42840
42840 42850
42850 42860
42860 42870
42870 42880
42880 42890
42890 42900
42900 42910
42910 42920
42920 42930
42930 42940
42940 42950
42950 42960
42960 42970
42970 42980
42980 42990
42990 43000
43000 43010
43010 43020
43020 43030
43030 43040
43040 43050
43050 43060
43060 43070
43070 43080
43080 43090
43090 43100
43100 43110
43110 43120
43120 43130
43130 43140
43140 43150
43150 43160
43160 43170
43170 43180
43180 43190
43190 43200
43200 43210
43210 43220
43220 43230
43230 43240
43240 43250
43250 43260
43260 43270
43270 43280
43280 43290
43290 43300
43300 43310
43310 43320
43320 43330
43330 43340
43340 43350
43350 43360
43360 43370
43370 43380
43380 43390
43390 43400
43400 43410
43410 43420
43420 43430
43430 43440
43440 43450
43450 43460
43460 43470
43470 43480
43480 43490
43490 43500
43500 43510
43510 43520
43520 43530
43530 43540
43540 43550
43550 43560
43560 43570
4357

49570 49580
49580 49590
49590 49600
49600 49610
49610 49620
49620 49630
49630 49640
49640 49650
49650 49660
49660 49670
49670 49680
49680 49690
49690 49700
49700 49710
49710 49720
49720 49730
49730 49740
49740 49750
49750 49760
49760 49770
49770 49780
49780 49790
49790 49800
49800 49810
49810 49820
49820 49830
49830 49840
49840 49850
49850 49860
49860 49870
49870 49880
49880 49890
49890 49900
49900 49910
49910 49920
49920 49930
49930 49940
49940 49950
49950 49960
49960 49970
49970 49980
49980 49990
49990 50000
50000 50010
50010 50020
50020 50030
50030 50040
50040 50050
50050 50060
50060 50070
50070 50080
50080 50090
50090 50100
50100 50110
50110 50120
50120 50130
50130 50140
50140 50150
50150 50160
50160 50170
50170 50180
50180 50190
50190 50200
50200 50210
50210 50220
50220 50230
50230 50240
50240 50250
50250 50260
50260 50270
50270 50280
50280 50290
50290 50300
50300 50310
50310 50320
50320 50330
50330 50340
50340 50350
50350 50360
50360 50370
50370 50380
50380 50390
50390 50400
5040

56400 56410
56410 56420
56420 56430
56430 56440
56440 56450
56450 56460
56460 56470
56470 56480
56480 56490
56490 56500
56500 56510
56510 56520
56520 56530
56530 56540
56540 56550
56550 56560
56560 56570
56570 56580
56580 56590
56590 56600
56600 56610
56610 56620
56620 56630
56630 56640
56640 56650
56650 56660
56660 56670
56670 56680
56680 56690
56690 56700
56700 56710
56710 56720
56720 56730
56730 56740
56740 56750
56750 56760
56760 56770
56770 56780
56780 56790
56790 56800
56800 56810
56810 56820
56820 56830
56830 56840
56840 56850
56850 56860
56860 56870
56870 56880
56880 56890
56890 56900
56900 56910
56910 56920
56920 56930
56930 56940
56940 56950
56950 56960
56960 56970
56970 56980
56980 56990
56990 57000
57000 57010
57010 57020
57020 57030
57030 57040
57040 57050
57050 57060
57060 57070
57070 57080
57080 57090
57090 57100
57100 57110
57110 57120
57120 57130
57130 57140
57140 57150
57150 57160
57160 57170
57170 57180
57180 57190
57190 57200
57200 57210
57210 57220
57220 57230
5723

63230 63240
63240 63250
63250 63260
63260 63270
63270 63280
63280 63290
63290 63300
63300 63310
63310 63320
63320 63330
63330 63340
63340 63350
63350 63360
63360 63370
63370 63380
63380 63390
63390 63400
63400 63410
63410 63420
63420 63430
63430 63440
63440 63450
63450 63460
63460 63470
63470 63480
63480 63490
63490 63500
63500 63510
63510 63520
63520 63530
63530 63540
63540 63550
63550 63560
63560 63570
63570 63580
63580 63590
63590 63600
63600 63610
63610 63620
63620 63630
63630 63640
63640 63650
63650 63660
63660 63670
63670 63680
63680 63690
63690 63700
63700 63710
63710 63720
63720 63730
63730 63740
63740 63750
63750 63760
63760 63770
63770 63780
63780 63790
63790 63800
63800 63810
63810 63820
63820 63830
63830 63840
63840 63850
63850 63860
63860 63870
63870 63880
63880 63890
63890 63900
63900 63910
63910 63920
63920 63930
63930 63940
63940 63950
63950 63960
63960 63970
63970 63980
63980 63990
63990 64000
64000 64010
64010 64020
64020 64030
64030 64040
64040 64050
64050 64060
6406

,id_valid_ean,name_valid_ean,id_invalid_ean,name_invalid_ean,similarity,SKU
0,64633037-3462-5232-b364-663039643861,CARNE de ALMUERZO PLUMROSE 340GR,36643338-3332-5331-b634-643834316134,Almuerzo carne de cerdo,0.777778,7591081130517
1,7baadd2b-deab-54fa-b066-ab8048ec7930,Carne Almuerzo Special Brand,36643338-3332-5331-b634-643834316134,Almuerzo carne de cerdo,0.777778,8682744523117
2,64616632-3133-5330-b664-656339356366,CALDO DE CARNE CARNE 19G,36643338-3332-5331-b634-643834316134,Almuerzo carne de cerdo,0.666667,7891000242452
3,61646535-3538-5338-b536-663332333337,Carne de cerdo porcionada,36643338-3332-5331-b634-643834316134,Almuerzo carne de cerdo,0.555556,7702129062015
4,32393664-6566-5264-a361-306263656131,Almuerzo parrilladas,36643338-3332-5331-b634-643834316134,Almuerzo carne de cerdo,0.555556,7700634001437
...,...,...,...,...,...,...
324065,b8864ea3-4898-41df-a319-bd82659a1bb3,1111122224126,62306661-3464-5966-a636-653533333930,"Metro tubo 1/2"" sp",1.000000,1111122224126
324066,60242153-274d-4808-b9ae-aeb8f9c2ede1,1111111111111,62306661-3464-5966-a636-653533333930,"Metro tubo 1/2"" sp",1.000000,7702045553239
324067,15834bee-bb3d-4e70-978a-78d74b4f7ce0,1111122018671,62306661-3464-5966-a636-653533333930,"Metro tubo 1/2"" sp",0.909091,1111122018671
324068,996e1ecf-7b42-4f06-9335-3129d8755750,1111128216613,62306661-3464-5966-a636-653533333930,"Metro tubo 1/2"" sp",0.909091,1111128216613


In [27]:
# Define la tabla que deseas crear en BigQuery
schema = [
    bigquery.SchemaField('id_valid_ean', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('name_valid_ean', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('id_invalid_ean', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('name_invalid_ean', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('SKU', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('similarity', 'NUMERIC', mode='REQUIRED'),
]
table = bigquery.Table(f'data-development-337318.data_standard.raw_standarized_products', schema=schema)

# Sube el DataFrame a BigQuery
job = client.load_table_from_dataframe(dfc, table)
job.result()

LoadJob<project=data-development-337318, location=us-west1, id=2e3b55ca-3c0b-49a6-9ac7-b18a090ca237>

In [4]:
#dfc.to_csv("All.csv", index=False)
dfc = pd.read_csv("All.csv")
dfc

,id_valid_ean,name_valid_ean,id_invalid_ean,name_invalid_ean,similarity,SKU
0,64633037-3462-5232-b364-663039643861,CARNE de ALMUERZO PLUMROSE 340GR,36643338-3332-5331-b634-643834316134,Almuerzo carne de cerdo,0.777778,7.591081e+12
1,7baadd2b-deab-54fa-b066-ab8048ec7930,Carne Almuerzo Special Brand,36643338-3332-5331-b634-643834316134,Almuerzo carne de cerdo,0.777778,8.682745e+12
2,64616632-3133-5330-b664-656339356366,CALDO DE CARNE CARNE 19G,36643338-3332-5331-b634-643834316134,Almuerzo carne de cerdo,0.666667,7.891000e+12
3,61646535-3538-5338-b536-663332333337,Carne de cerdo porcionada,36643338-3332-5331-b634-643834316134,Almuerzo carne de cerdo,0.555556,7.702129e+12
4,32393664-6566-5264-a361-306263656131,Almuerzo parrilladas,36643338-3332-5331-b634-643834316134,Almuerzo carne de cerdo,0.555556,7.700634e+12
...,...,...,...,...,...,...
324068,b8864ea3-4898-41df-a319-bd82659a1bb3,1111122224126,62306661-3464-5966-a636-653533333930,"Metro tubo 1/2"" sp",1.000000,1.111122e+12
324069,60242153-274d-4808-b9ae-aeb8f9c2ede1,1111111111111,62306661-3464-5966-a636-653533333930,"Metro tubo 1/2"" sp",1.000000,7.702046e+12
324070,15834bee-bb3d-4e70-978a-78d74b4f7ce0,1111122018671,62306661-3464-5966-a636-653533333930,"Metro tubo 1/2"" sp",0.909091,1.111122e+12
324071,996e1ecf-7b42-4f06-9335-3129d8755750,1111128216613,62306661-3464-5966-a636-653533333930,"Metro tubo 1/2"" sp",0.909091,1.111128e+12


In [44]:
dfg = dfc.groupby(by=["id_invalid_ean", "name_invalid_ean", "SKU"]).count().copy()
dfg

id_valid_ean  \
id_invalid_ean                       name_invalid_ean                  SKU                           
0004601b-7707-5c05-a15e-e89f41a48482 Granizado Premium - Café 9 oz     2022039999573             1   
                                                                       6916861011076             1   
                                                                       7591052000719             1   
                                                                       7861087702781             1   
                                                                       7908339907713             1   
...                                                                                            ...   
zyxvqg1eykjvk254                     Una pantalla x mes de Disney plus 7501030426905             1   
                                                                       7506195124512             1   
                                                                       7506295326564             1   
                                                                       7506309873701             1   
                                                                       7506339319033             1   

                                                                                      name_valid_ean  \
id_invalid_ean                       name_invalid_ean                  SKU                             
0004601b-7707-5c05-a15e-e89f41a48482 Granizado Premium - Café 9 oz     2022039999573               1   
                                                                       6916861011076               1   
                                                                       7591052000719               1   
                                                                       7861087702781               1   
                                                                       7908339907713               1   
...                                                                                              ...   
zyxvqg1eykjvk254                     Una pantalla x mes de Disney plus 7501030426905               1   
                                                                       7506195124512               1   
                                                                       7506295326564               1   
                                                                       7506309873701               1   
                                                                       7506339319033               1   

                                                                                      similarity  
id_invalid_ean                       name_invalid_ean                  SKU                        
0004601b-7707-5c05-a15e-e89f41a48482 Granizado Premium - Café 9 oz     2022039999573           1  
                                                                       6916861011076           1  
                                                                       7591052000719           1  
                                                                       7861087702781           1  
                                                                       7908339907713           1  
...                                                                                          ...  
zyxvqg1eykjvk254                     Una pantalla x mes de Disney plus 7501030426905           1  
                                                                       7506195124512           1  
                                                                       7506295326564           1  
                                                                       7506309873701           1  
                                                                       7506339319033           1  

[266341 rows x 3 columns]

In [45]:
dfg = dfg[["id_valid_ean"]]
dfg.rename(columns={"id_valid_ean": "count"},inplace=True)
dfg

/var/folders/fc/1mlb4bvs6qsc662f5v5ck7jh0000gn/T/ipykernel_90678/264515409.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfg.rename(columns={"id_valid_ean": "count"},inplace=True)


count
id_invalid_ean                       name_invalid_ean                  SKU                 
0004601b-7707-5c05-a15e-e89f41a48482 Granizado Premium - Café 9 oz     2022039999573      1
                                                                       6916861011076      1
                                                                       7591052000719      1
                                                                       7861087702781      1
                                                                       7908339907713      1
...                                                                                     ...
zyxvqg1eykjvk254                     Una pantalla x mes de Disney plus 7501030426905      1
                                                                       7506195124512      1
                                                                       7506295326564      1
                                                                       7506309873701      1
                                                                       7506339319033      1

[266341 rows x 1 columns]

In [46]:
dfg.reset_index(inplace=True)
dfg

,id_invalid_ean,name_invalid_ean,SKU,count
0,0004601b-7707-5c05-a15e-e89f41a48482,Granizado Premium - Café 9 oz,2022039999573,1
1,0004601b-7707-5c05-a15e-e89f41a48482,Granizado Premium - Café 9 oz,6916861011076,1
2,0004601b-7707-5c05-a15e-e89f41a48482,Granizado Premium - Café 9 oz,7591052000719,1
3,0004601b-7707-5c05-a15e-e89f41a48482,Granizado Premium - Café 9 oz,7861087702781,1
4,0004601b-7707-5c05-a15e-e89f41a48482,Granizado Premium - Café 9 oz,7908339907713,1
...,...,...,...,...
266336,zyxvqg1eykjvk254,Una pantalla x mes de Disney plus,7501030426905,1
266337,zyxvqg1eykjvk254,Una pantalla x mes de Disney plus,7506195124512,1
266338,zyxvqg1eykjvk254,Una pantalla x mes de Disney plus,7506295326564,1
266339,zyxvqg1eykjvk254,Una pantalla x mes de Disney plus,7506309873701,1


In [73]:
pids = dfg.id_invalid_ean.unique()

sdf = pd.DataFrame(columns=dfg.columns)
ref = 0
for pid in pids:
    tdf = dfg[dfg["id_invalid_ean"] == pid]
    idx = tdf['count'].idxmax()
    max_row = tdf.loc[idx]
    sdf = sdf.append(max_row, ignore_index=True)
    
    if ref%1000==0:
        print(f"Registro {ref} de {len(pids)}")
    
    ref += 1
sdf

/var/folders/fc/1mlb4bvs6qsc662f5v5ck7jh0000gn/T/ipykernel_90678/4270321916.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdf = sdf.append(max_row, ignore_index=True)


Registro 0 de 64814
Registro 1000 de 64814
Registro 2000 de 64814
Registro 3000 de 64814
Registro 4000 de 64814
Registro 5000 de 64814
Registro 6000 de 64814
Registro 7000 de 64814
Registro 8000 de 64814
Registro 9000 de 64814
Registro 10000 de 64814
Registro 11000 de 64814
Registro 12000 de 64814
Registro 13000 de 64814
Registro 14000 de 64814
Registro 15000 de 64814
Registro 16000 de 64814
Registro 17000 de 64814
Registro 18000 de 64814
Registro 19000 de 64814
Registro 20000 de 64814
Registro 21000 de 64814
Registro 22000 de 64814
Registro 23000 de 64814
Registro 24000 de 64814
Registro 25000 de 64814
Registro 26000 de 64814
Registro 27000 de 64814
Registro 28000 de 64814
Registro 29000 de 64814
Registro 30000 de 64814
Registro 31000 de 64814
Registro 32000 de 64814
Registro 33000 de 64814
Registro 34000 de 64814
Registro 35000 de 64814
Registro 36000 de 64814
Registro 37000 de 64814
Registro 38000 de 64814
Registro 39000 de 64814
Registro 40000 de 64814
Registro 41000 de 64814
Regis

,id_invalid_ean,name_invalid_ean,SKU,count
0,0004601b-7707-5c05-a15e-e89f41a48482,Granizado Premium - Café 9 oz,2022039999573,1
1,0005c5e7-ff72-5762-bc3e-332acaafe7e9,GOMAS SPLASH RELLENAS,7702993031117,5
2,0005cf16-a73d-5764-8829-5062d949bfb4,Pastillero Escarchado x6,5447505478517,1
3,0005f071-4c75-5198-b843-1219b315721b,Gatorade 500ml surtido,7702192422051,2
4,0006803f-e243-5857-ad5f-06a4208f07fc,Piña Colada,6973448977811,1
...,...,...,...,...
64809,zxanp0x7b04mqgx4,Fruco mayonesa 85g,7702047038734,3
64810,zxml0yadgug1520v,Jabón Especializado,7702031414346,2
64811,zxu5wjbwq8kume27,Jean Paul Gaultier ( Le Male ),3423474773954,1
64812,zyp3twd44dmlqwiw,Tornillo Cabeza 8 6x25 Cromado (*300)(101)(301),7707036900191,1


In [3]:
#sdf.to_csv("max_count.csv",index=False)
sdf = pd.read_csv("max_count.csv")
sdf

,id_invalid_ean,name_invalid_ean,SKU,count
0,0004601b-7707-5c05-a15e-e89f41a48482,Granizado Premium - Café 9 oz,2022039999573,1
1,0005c5e7-ff72-5762-bc3e-332acaafe7e9,GOMAS SPLASH RELLENAS,7702993031117,5
2,0005cf16-a73d-5764-8829-5062d949bfb4,Pastillero Escarchado x6,5447505478517,1
3,0005f071-4c75-5198-b843-1219b315721b,Gatorade 500ml surtido,7702192422051,2
4,0006803f-e243-5857-ad5f-06a4208f07fc,Piña Colada,6973448977811,1
...,...,...,...,...
64809,zxanp0x7b04mqgx4,Fruco mayonesa 85g,7702047038734,3
64810,zxml0yadgug1520v,Jabón Especializado,7702031414346,2
64811,zxu5wjbwq8kume27,Jean Paul Gaultier ( Le Male ),3423474773954,1
64812,zyp3twd44dmlqwiw,Tornillo Cabeza 8 6x25 Cromado (*300)(101)(301),7707036900191,1


In [117]:
sdf["count"].value_counts()

1    34292
2    17414
3     7126
4     3449
5     2533
Name: count, dtype: int64

In [12]:
dft = pd.merge(sdf[sdf["count"] >= 2], dfc, on=["id_invalid_ean", "SKU"], how="left").drop_duplicates(subset=["id_invalid_ean", "SKU"]).reset_index(drop=True).drop(columns=["name_invalid_ean_y"])
dft

,id_invalid_ean,name_invalid_ean_x,SKU,count,id_valid_ean,name_valid_ean,similarity
0,0005c5e7-ff72-5762-bc3e-332acaafe7e9,GOMAS SPLASH RELLENAS,7702993031117,5,63626336-3238-5062-b166-633738376466,Splash Rellenas,0.750000
1,0005f071-4c75-5198-b843-1219b315721b,Gatorade 500ml surtido,7702192422051,2,33333666-6130-5261-a261-336336626633,Gatorade Surtido 500 ML,1.000000
2,0008c12f-375a-5798-b96a-9a9ccab8f8d8,DOLICOX FORTE UND,7703712014251,4,63356665-6563-5366-b537-616130366166,Dolicox forte tabletas max coaspharma,1.000000
3,000fd560-aace-5655-81da-050293021235,club dorada,7702004003454,2,79707975-d95b-44bb-850d-b7eb09144ffd,Club dorada,1.000000
4,0010f92f-f3f6-5032-a91b-e202beff8640,Arepa de Salchicha Ranchera,7701101358245,4,61455cd3-2297-53b3-b4ea-a27aaf2c4f45,SALCHICHA RANCHERA x 5und,1.000000
...,...,...,...,...,...,...,...
30517,zlj5yw329euzo8hr,Costeñita Bacana 320Ml Botella,7702004005649,5,zlj5yw329euzo8hr,Costeñita Bacana 320Ml Botella,1.000000
30518,zlmwivb0ldu04sfs,Empanadas Mixta,7709990386455,2,32373038-3134-5765-a436-373666386234,Masa para empanadas Rey,0.714286
30519,ztb0x55zen37k4w0,Tapabocas quirúrgicos caja x 50 unidades,8936119380684,2,61336664-6638-5639-b938-353033666239,Tapabocas quirúrgicos Nut,0.900000
30520,zxanp0x7b04mqgx4,Fruco mayonesa 85g,7702047038734,3,c7bbe157-eaa7-5ffd-9bb0-1e0f1855f6dc,Mayonesa Fruco 85gr,1.000000


In [16]:
dft = dft[(dft['similarity'] <= 1) & (dft['count'] >= 2)].copy()
dft.reset_index(drop=True, inplace=True)
dft

,id_invalid_ean,name_invalid_ean_x,SKU,count,id_valid_ean,name_valid_ean,similarity
0,0005c5e7-ff72-5762-bc3e-332acaafe7e9,GOMAS SPLASH RELLENAS,7702993031117,5,63626336-3238-5062-b166-633738376466,Splash Rellenas,0.750000
1,0005f071-4c75-5198-b843-1219b315721b,Gatorade 500ml surtido,7702192422051,2,33333666-6130-5261-a261-336336626633,Gatorade Surtido 500 ML,1.000000
2,0008c12f-375a-5798-b96a-9a9ccab8f8d8,DOLICOX FORTE UND,7703712014251,4,63356665-6563-5366-b537-616130366166,Dolicox forte tabletas max coaspharma,1.000000
3,000fd560-aace-5655-81da-050293021235,club dorada,7702004003454,2,79707975-d95b-44bb-850d-b7eb09144ffd,Club dorada,1.000000
4,0010f92f-f3f6-5032-a91b-e202beff8640,Arepa de Salchicha Ranchera,7701101358245,4,61455cd3-2297-53b3-b4ea-a27aaf2c4f45,SALCHICHA RANCHERA x 5und,1.000000
...,...,...,...,...,...,...,...
27099,zlj5yw329euzo8hr,Costeñita Bacana 320Ml Botella,7702004005649,5,zlj5yw329euzo8hr,Costeñita Bacana 320Ml Botella,1.000000
27100,zlmwivb0ldu04sfs,Empanadas Mixta,7709990386455,2,32373038-3134-5765-a436-373666386234,Masa para empanadas Rey,0.714286
27101,ztb0x55zen37k4w0,Tapabocas quirúrgicos caja x 50 unidades,8936119380684,2,61336664-6638-5639-b938-353033666239,Tapabocas quirúrgicos Nut,0.900000
27102,zxanp0x7b04mqgx4,Fruco mayonesa 85g,7702047038734,3,c7bbe157-eaa7-5ffd-9bb0-1e0f1855f6dc,Mayonesa Fruco 85gr,1.000000


In [31]:
dft = dft[["id_invalid_ean", "name_invalid_ean_x", "SKU", "name_valid_ean", "id_valid_ean", "count", "similarity"]].copy()
dft = dft.rename(columns={"id_invalid_ean":"product_id", "name_invalid_ean_x": "name", "SKU": "ean13", "name_valid_ean": "similar_product", "id_valid_ean":"similar_product_id"})

In [32]:
dft

,product_id,name,ean13,similar_product,similar_product_id,count,similarity
0,0005c5e7-ff72-5762-bc3e-332acaafe7e9,GOMAS SPLASH RELLENAS,7702993031117,Splash Rellenas,63626336-3238-5062-b166-633738376466,5,0.750000
1,0005f071-4c75-5198-b843-1219b315721b,Gatorade 500ml surtido,7702192422051,Gatorade Surtido 500 ML,33333666-6130-5261-a261-336336626633,2,1.000000
2,0008c12f-375a-5798-b96a-9a9ccab8f8d8,DOLICOX FORTE UND,7703712014251,Dolicox forte tabletas max coaspharma,63356665-6563-5366-b537-616130366166,4,1.000000
3,000fd560-aace-5655-81da-050293021235,club dorada,7702004003454,Club dorada,79707975-d95b-44bb-850d-b7eb09144ffd,2,1.000000
4,0010f92f-f3f6-5032-a91b-e202beff8640,Arepa de Salchicha Ranchera,7701101358245,SALCHICHA RANCHERA x 5und,61455cd3-2297-53b3-b4ea-a27aaf2c4f45,4,1.000000
...,...,...,...,...,...,...,...
27099,zlj5yw329euzo8hr,Costeñita Bacana 320Ml Botella,7702004005649,Costeñita Bacana 320Ml Botella,zlj5yw329euzo8hr,5,1.000000
27100,zlmwivb0ldu04sfs,Empanadas Mixta,7709990386455,Masa para empanadas Rey,32373038-3134-5765-a436-373666386234,2,0.714286
27101,ztb0x55zen37k4w0,Tapabocas quirúrgicos caja x 50 unidades,8936119380684,Tapabocas quirúrgicos Nut,61336664-6638-5639-b938-353033666239,2,0.900000
27102,zxanp0x7b04mqgx4,Fruco mayonesa 85g,7702047038734,Mayonesa Fruco 85gr,c7bbe157-eaa7-5ffd-9bb0-1e0f1855f6dc,3,1.000000


In [33]:
# Define la tabla que deseas crear en BigQuery
schema = [
    bigquery.SchemaField('product_id', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('name', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('ean13', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('similar_product', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('similar_product_id', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('count', 'NUMERIC', mode='REQUIRED'),
    bigquery.SchemaField('similarity', 'NUMERIC', mode='REQUIRED'),
]
table = bigquery.Table(f'data-development-337318.data_standard.clean_standarized_products', schema=schema)

# Sube el DataFrame a BigQuery
job = client.load_table_from_dataframe(dft, table)
job.result()

LoadJob<project=data-development-337318, location=us-west1, id=f4e443c5-bd0b-404c-ad56-f3b6a69d7575>

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import psutil

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sentencepiece import SentencePieceProcessor
from logging import getLogger
from typing import List
import os
import numpy as np
from tqdm import tqdm

lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words('spanish'))
NB_PHYSICAL_CORES = psutil.cpu_count()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=NB_PHYSICAL_CORES - 2)

import spacy
nlp = spacy.load("es_core_news_sm")

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [65]:
## read valid ean data
query = f"""SELECT * FROM `data-development-337318.data_standard.clean_standarized_products`  WHERE length(CAST(ean13 AS STRING)) = 13"""
df = pd.read_gbq(query, dialect='standard', use_bqstorage_api=True, project_id="data-development-337318")
df.head()

,product_id,name,ean13,similar_product,similar_product_id,count,similarity
0,63643737-6137-5837-a436-613863383336,Aceite cocina frasco x 350 ML,7773103000064,Aceite Fino Mezcla Frasco x 900 ml,1d1fa241-15b5-540d-b2c9-00d8f2c1c3fa,2,0.70
1,97fsbqydbbdzz0xr,Doritos 43 g,7790310984192,doritos 85,3af05ab6-9e0e-5c47-be6b-d0e4545d263c,2,1.00
2,34a06903-1f93-57ca-9baf-9d004d4d6d36,HILO ELASTICO MTR,7506305163264,Hilo Elastico Nylon 0.5mm 25mt,431ff92e-060a-555a-a3f4-9a2236892a85,2,0.75
3,b2210b1e-4884-5c6d-983a-64467f6f6479,Removedor de callo 120 ml,7707313842176,Removedor callo grande,65396233-3936-5564-a361-666138343334,2,1.00
4,34363663-3135-5932-b565-346439303030,Bolichetto,7862124230144,Bolichito 15g,32326365-6465-5632-a630-386333663531,2,0.75


In [66]:
import re
import unicodedata
import spacy

REMOVE_WORDS = ['x', 'grande', 'sabor','caja','tableta','lata','color','kit', 'unid', 'mediano', 'colores', 'tab',
                'mini', 'und', 'unidade', 'gr','unidad', 'grs', 'tabletas', 'doble', 'unidades', 'gramo', 'c','bolsa', 'pequeño']
REMOVE_PATTERN = r'\b(?:{})\b'.format('|'.join(map(re.escape, REMOVE_WORDS)))

def remove_measure_units(text):
    patern = r'\b\d+\s*(gramos|gr|g|kg|kilogramo|kilogramos|l|lt|kit|litro|litros|ml|mililitro|mililitros|mg|miligramo|grande|pequeño|pequeno|mini|kit|plus|x|X|pg|talla|hojas|unidades)\b'
    result = re.sub(patern, '', text)
    return result

def remove_colors(text):
    patern = r'\b(rojo|azul|verde|amarillo|naranja|morado|rosa|marrón|negro|blanco)\b'
    result = re.sub(patern, '', text, flags=re.IGNORECASE)
    return result

def remove_words(text, pattern=REMOVE_PATTERN):
    result = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return result

def remove_diacritics(text):
    normalized_text = unicodedata.normalize('NFD', text)
    removed_diacritics = ''.join(c for c in normalized_text if not unicodedata.combining(c))
    return removed_diacritics

def normalize_text_funtion(text: str):
    text = text.lower()
    text = text.replace('[^\w\s]', '')
    text = nltk.word_tokenize(text)
    text = [item for item in text if item not in stop]
    text = [lemmatizer.lemmatize(y) for y in text]
    text = ' '.join(text)
    text = [y.lemma_ for y in nlp(text)]
    text = ' '.join(text)
    text = remove_measure_units(text)
    text = remove_colors(text)
    text = remove_words(text)
    text = remove_diacritics(text)
    return text

# df['name_clean'] = df['name_clean'].parallel_apply(lambda x: remove_measure_units(x))
# df['name_clean'] = df['name_clean'].parallel_apply(lambda x: remove_colors(x))
# df['name_clean'] = df['name_clean'].parallel_apply(lambda x: remove_words(x))
# df['name_clean'] = df['name_clean'].parallel_apply(lambda x: remove_diacritics(x))

In [67]:
df.loc[:, 'clean_name'] = df.name.parallel_apply(normalize_text_funtion)

In [68]:
import re

def tokenize(text):
    tokens = []
    for match in re.finditer(r"\w+", text):
        tokens.append(match.group())

    return tokens

In [69]:
df.loc[:, 'vector_name'] = df.clean_name.parallel_apply(tokenize)
df

,product_id,name,ean13,similar_product,similar_product_id,count,similarity,clean_name,vector_name
0,63643737-6137-5837-a436-613863383336,Aceite cocina frasco x 350 ML,7773103000064,Aceite Fino Mezcla Frasco x 900 ml,1d1fa241-15b5-540d-b2c9-00d8f2c1c3fa,2,0.700000,aceite cocin frasco,"[aceite, cocin, frasco]"
1,97fsbqydbbdzz0xr,Doritos 43 g,7790310984192,doritos 85,3af05ab6-9e0e-5c47-be6b-d0e4545d263c,2,1.000000,dorito,[dorito]
2,34a06903-1f93-57ca-9baf-9d004d4d6d36,HILO ELASTICO MTR,7506305163264,Hilo Elastico Nylon 0.5mm 25mt,431ff92e-060a-555a-a3f4-9a2236892a85,2,0.750000,hilo elastico mtr,"[hilo, elastico, mtr]"
3,b2210b1e-4884-5c6d-983a-64467f6f6479,Removedor de callo 120 ml,7707313842176,Removedor callo grande,65396233-3936-5564-a361-666138343334,2,1.000000,removedor callo,"[removedor, callo]"
4,34363663-3135-5932-b565-346439303030,Bolichetto,7862124230144,Bolichito 15g,32326365-6465-5632-a630-386333663531,2,0.750000,bolichetto,[bolichetto]
...,...,...,...,...,...,...,...,...,...
26398,30739517-8a51-52fd-bc8a-76f50083e5eb,ACONDICIONADOR CABELLO SECO MAGIC,7908132266239,Acondicionador Cabello seco 300 ml,34343362-3237-5234-b138-623635323633,5,0.909091,acondicionador cabello seco magic,"[acondicionador, cabello, seco, magic]"
26399,63653233-6634-5863-b030-323864303764,Guayaproff jarabe,7703038011903,GUAYAPROFF JARABE UNIDAD 120ML,f7aa8825-4b39-531e-8585-93d1a8590a8e,5,1.000000,guayaproff jarabe,"[guayaproff, jarabe]"
26400,911c652e-36bd-4407-a875-f8e59fb67190,Guayaprof,7703038011903,Guayaprof,911c652e-36bd-4407-a875-f8e59fb67190,5,1.000000,guayaprof,[guayaprof]
26401,30656566-3530-5334-b233-393435333836,Vino abocado 2 litros,7702121234175,Vino Abocado Casa grajales,10bdf8d1-53de-5e52-870c-ec311501b7e8,5,0.857143,venir abocado,"[venir, abocado]"


In [70]:
import collections

def get_frequency_words(list_of_arrays):
    """Gets the frequency words of a group of arrays.

    Args:
    list_of_arrays: A list of lists of words.

    Returns:
    A dictionary of words to frequencies.
    """

    word_counts = collections.defaultdict(int)
    for array in list_of_arrays:
        for word in array:
            word_counts[word] += 1

    return dict(word_counts)

In [71]:
ean_unique = list(df.ean13.unique())

frq = {}
for ean in tqdm(ean_unique):
    array = df[df.ean13 == ean].vector_name.to_list()
    frq[ean] = get_frequency_words(array)
frq

100%|███████████████████████████████████| 8737/8737 [00:02<00:00, 3631.35it/s]


{7773103000064: {'aceite': 2, 'cocin': 1, 'frasco': 2},
 7790310984192: {'dorito': 1},
 7506305163264: {'hilo': 1, 'elastico': 1, 'mtr': 1},
 7707313842176: {'removedor': 1, 'callo': 1},
 7862124230144: {'bolichetto': 1},
 7591016873472: {'samba': 1},
 7908132274944: {'fruto': 1},
 7861065503744: {'rizado': 1, 'pollo': 1},
 7702152107264: {'rico': 3, 'original': 4, 'todorico': 1},
 7861065504512: {'anillo': 3, 'plata': 1, 'cebollar': 2},
 7753749001984: {'cristal': 1},
 7501095467264: {'pan': 1, 'uva': 1},
 7793450000128: {'pasaboca': 1,
  'hojaldre': 7,
  'champinon': 1,
  'margarina': 1,
  'lb': 2,
  'dedito': 1,
  'pan': 1,
  'hojaldrado': 1,
  'rollo': 1,
  'hawaiano': 1,
  'deliojaldre': 1},
 7707351430144: {'leche': 1, 'santo': 1, 'domingo': 1, 'lt': 1},
 7702210001152: {'crema': 1, 'desengrasante': 1, 't': 1},
 7703202221312: {'buscapina': 1,
  'nf': 2,
  'compositum': 2,
  '325': 1,
  'pasta': 1,
  'buscapin': 1},
 7702007046144: {'burbujo': 1,
  'burbuja': 16,
  'jet': 14,
  '

In [72]:
pdf = pd.DataFrame(columns=["ean13", "word", "freq"])
idx = 0
for fk,fv in frq.items():
    for ik, iv in fv.items():
        pdf.loc[idx] = [fk, ik, iv]
        idx += 1
pdf

,ean13,word,freq
0,7773103000064,aceite,2
1,7773103000064,cocin,1
2,7773103000064,frasco,2
3,7790310984192,dorito,1
4,7506305163264,hilo,1
...,...,...,...
33041,7908132266239,seco,1
33042,7908132266239,magic,1
33043,7703038011903,guayaproff,1
33044,7703038011903,jarabe,1


In [85]:
spdf = pdf[(pdf.freq > 1)].copy()
spdf

,ean13,word,freq
0,7773103000064,aceite,2
2,7773103000064,frasco,2
14,7702152107264,rico,3
15,7702152107264,original,4
17,7861065504512,anillo,3
...,...,...,...
33012,7702402054393,bosque,2
33021,7707264570876,nucherol,2
33029,7705137110013,agua,3
33030,7705137110013,oxigenado,3


In [86]:
spdf = spdf.sort_values(by=["ean13", "freq"]).reset_index(drop=True)
spdf

,ean13,word,freq
0,1003010033333,agua,2
1,1007084701235,monster,4
2,1007084703475,monster,3
3,1007084703536,monster,2
4,1012127427799,alisadora,2
...,...,...,...
8085,9780825414084,pastor,2
8086,9780851517995,piedra,3
8087,9788499393353,b93,2
8088,9789587304886,colorear,2


In [87]:
spdf

,ean13,word,freq
0,1003010033333,agua,2
1,1007084701235,monster,4
2,1007084703475,monster,3
3,1007084703536,monster,2
4,1012127427799,alisadora,2
...,...,...,...
8085,9780825414084,pastor,2
8086,9780851517995,piedra,3
8087,9788499393353,b93,2
8088,9789587304886,colorear,2


In [88]:
len(spdf.ean13.unique())

3145

In [92]:
# Define la tabla que deseas crear en BigQuery
schema = [
    bigquery.SchemaField('ean13', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('word', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('freq', 'NUMERIC', mode='REQUIRED')
]
table = bigquery.Table(f'data-development-337318.data_standard.test', schema=schema)

# Sube el DataFrame a BigQuery
job = client.load_table_from_dataframe(spdf, table)
job.result()

LoadJob<project=data-development-337318, location=us-west1, id=5e516f80-8b77-4b85-a63b-a7ee325a729e>